In [106]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from datetime import datetime as dt
from sklearn.model_selection import train_test_split
import catboost
from catboost import CatBoostClassifier

# Read Data

In [107]:
df_credits = pd.read_csv('../data/User Credits Student Access.csv', encoding='utf-8')
df_atlas = pd.read_csv('../data/Atlas Cechu Student Access.csv', encoding='utf-8')
df_payments = pd.read_csv('../data/Payments Student Access.csv', encoding='utf-8')

In [108]:
df_atlas.columns

Index(['user_id', 'all_intro-2022', 'atlas_vzorek-2023', 'atlas_vzorek-2024',
       'all_intro-trendaro_panel_all', 'sex-woman', 'sex-man', 'age-15_17',
       'age-18_24', 'age-25_34',
       ...
       'check-tv', 'check-radio', 'check-household', 'check-overdraft',
       'check-naramek', 'isic2_age-21_26', 'isic2_age-15_20', 'isic_age-15_17',
       'isic_age-18_22', 'isic_age-23_26'],
      dtype='object', length=1121)

In [133]:
df_credits

,user,credits,is_active,is_verified,is_locked,wage
0,STUD66006,25,1,1,0,Peníze
1,STUD22095,51,1,1,0,Peníze
2,STUD77411,0,1,1,1,Peníze
3,STUD56329,0,1,0,0,NaN
4,STUD23516,30,1,1,0,Peníze
...,...,...,...,...,...,...
89906,STUD54678,789,1,1,0,Peníze
89907,STUD43709,0,0,1,0,NaN
89908,STUD21881,0,0,0,0,NaN
89909,STUD72037,16,1,1,0,Lékaři bez hranic


# Negative Values investigation & Chekcing other things

In [109]:
df_c_negative = df_credits[df_credits['credits']<0]
df_credits[df_credits.user.isin(df_c_negative.user) == True]

# IDK what I'm doing with payments
df_payments[df_payments.user.isin(df_c_negative.user)==True].sort_values(['user','created_at'])
df_payments.state.unique()
df_payments[df_payments.user=='STUD54678']
df_credits[df_credits.user=='STUD54678']

,user,credits,is_active,is_verified,is_locked,wage
89906,STUD54678,789,1,1,0,Peníze


In [110]:
df_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30087 entries, 0 to 30086
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          30086 non-null  float64
 1   created_at  30087 non-null  object 
 2   changed_at  30087 non-null  object 
 3   user        27652 non-null  object 
 4   batch       27502 non-null  object 
 5   credits     30086 non-null  float64
 6   state       30086 non-null  object 
 7   type        30086 non-null  object 
dtypes: float64(2), object(6)
memory usage: 1.8+ MB


# Data Cleaning

In [111]:
df_credits_cleaned = df_credits[df_credits['credits']>0]
df_payments_cleaned = df_payments[df_payments['id'].notna()]

# What is core table for merge? - Payments // Can't I just use inner join? Therefore I dont' have to work with nul values - I don't want them in model anyway

In [112]:
#I have an assumption that all users should be in Atlas Cechu. However, after cheking my assumption it tourns out not to be true. 

# Convert the boolean series to a DataFrame with a column name
atlas_in_credits = df_credits_cleaned.user.isin(df_atlas.user_id).rename('User in Atlas')
atlas_in_payments = df_payments_cleaned.user.isin(df_atlas.user_id)

# Plotting
fig_ac = px.bar(atlas_in_credits.value_counts(), title='How many users from Atlas Cechu are in the credits table', labels={'index': 'User in Atlas (True/False)', 'value': 'Count'})
fig_ap = px.bar(atlas_in_payments.value_counts(), title='How many users from Atlas Cechu are in the payments table', labels={'index': 'User in Atlas (True/False)', 'value': 'Count'})

fig_ac.show()
fig_ap.show()

In [113]:
df_atlas.user_id.isin(df_credits_cleaned.user).value_counts()

user_id
True     40672
False     2861
Name: count, dtype: int64

In [114]:
df_atlas.user_id.isin(df_payments_cleaned.user).value_counts()

user_id
False    34824
True      8709
Name: count, dtype: int64

Since we want to find out when who pulls what, it makes sense to use the payments table and connect data to it - even though there are users in payments tables that are not in Atlas Cechu

In [115]:
#df_merge_credits = pd.merge(df_payments_cleaned, df_credits_cleaned, how="inner", left_on='user', right_on='user')
#df_merge_credits[df_merge_credits.user=='STUD92308']

"""
I don't find credits table to be interesting - mainly we cannot join them together due to different "time type" -- credits shows the newest version of creadits without historical changes, on the other hand table payments shows history of all transactions
"""

'\nI don\'t find credits table to be interesting - mainly we cannot join them together due to different "time type" -- credits shows the newest version of creadits without historical changes, on the other hand table payments shows history of all transactions\n'

In [116]:
df_merge_full = pd.merge(df_payments_cleaned, df_atlas, how='inner', left_on='user', right_on='user_id')
df_merge_full.rename(columns={'credits_x':'credits_payments', 'credits_y':'credits_credits'})
df_merge_full = pd.get_dummies(df_merge_full, columns=['type'], drop_first=True)
df_merge_full = df_merge_full.astype({col: int for col in df_merge_full.select_dtypes(include='bool').columns})

df_merge_full = df_merge_full[df_merge_full['state']!='CANCELLED']

# df_merge_full.to_csv('../data_output/df_merge_full.csv')

In [117]:
##### In the dataset we found out that in payments dataset values that are being cancelled has creadits = 0 and NaN user having credit 430 and still being PAID. However, that user doesn't have user_id

df_payments[df_payments['credits']<500].groupby('state').size()

state
CANCELLED    2000
PAID            1
dtype: int64

In [137]:
X -> atlas, y -> ?

,id,changed_at,user,batch,credits,state,user_id,all_intro-2022,atlas_vzorek-2023,atlas_vzorek-2024,...,isic_age-23_26,type_HANDIPET,type_MEDICINS_SANS_FRONTIERS,type_MONEY,type_TRENDARO,day,month,year,hour,weekday
0,30116.0,2025-04-07 10:32:05.073633+00:00,STUD83031,NaN,509.0,REQUEST,STUD83031,1,0,0,...,0,0,0,1,0,7,4,2025,10,0
1,30115.0,2025-04-07 10:30:09.304196+00:00,STUD356,NaN,538.0,REQUEST,STUD356,1,0,1,...,0,0,0,1,0,7,4,2025,10,0
2,30114.0,2025-04-07 09:57:04.343985+00:00,STUD25955,NaN,542.0,REQUEST,STUD25955,1,0,0,...,0,0,0,1,0,7,4,2025,9,0
3,30113.0,2025-04-07 09:15:39.069910+00:00,STUD45618,NaN,501.0,REQUEST,STUD45618,1,0,0,...,0,0,0,1,0,7,4,2025,9,0
4,30112.0,2025-04-07 08:28:47.838578+00:00,STUD44458,NaN,504.0,REQUEST,STUD44458,1,0,0,...,0,0,0,1,0,7,4,2025,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26874,36.0,2024-01-10 17:31:11.782802+00:00,STUD49275,5,506.0,PAID,STUD49275,1,0,0,...,0,0,0,1,0,11,8,2017,14,4
26875,35.0,2024-01-10 17:31:11.801715+00:00,STUD53465,5,523.0,PAID,STUD53465,1,0,0,...,0,0,0,1,0,11,8,2017,14,4
26876,34.0,2024-01-10 17:31:11.820777+00:00,STUD29324,5,511.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,11,8,2017,14,4
26877,31.0,2024-01-10 17:31:11.867765+00:00,STUD92308,NaN,511.0,PAID,STUD92308,1,1,0,...,0,0,1,0,0,10,8,2017,21,3


In [139]:
df_merge_full[df_merge_full['user']=='STUD29324']

,id,changed_at,user,batch,credits,state,user_id,all_intro-2022,atlas_vzorek-2023,atlas_vzorek-2024,...,isic_age-23_26,type_HANDIPET,type_MEDICINS_SANS_FRONTIERS,type_MONEY,type_TRENDARO,day,month,year,hour,weekday
2884,27204.0,2025-01-30 11:13:06.272277+00:00,STUD29324,280,507.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,29,1,2025,11,2
6976,23047.0,2024-10-10 09:37:21.336291+00:00,STUD29324,255,628.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,4,10,2024,12,4
11004,18828.0,2024-05-02 07:45:59.753111+00:00,STUD29324,224,505.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,1,5,2024,11,2
12715,17042.0,2024-02-15 16:54:29.290732+00:00,STUD29324,216,573.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,7,2,2024,15,2
20715,8238.0,2024-01-10 17:28:38.772490+00:00,STUD29324,131,514.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,11,3,2022,7,4
22265,6332.0,2024-01-10 17:29:16.050685+00:00,STUD29324,110,515.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,30,7,2021,7,4
23253,5018.0,2024-01-10 17:29:38.609722+00:00,STUD29324,90,514.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,11,12,2020,10,4
25143,2481.0,2024-01-10 17:30:26.241582+00:00,STUD29324,58,503.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,11,12,2019,7,2
26288,863.0,2024-01-10 17:30:56.065601+00:00,STUD29324,37,506.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,17,2,2019,18,6
26876,34.0,2024-01-10 17:31:11.820777+00:00,STUD29324,5,511.0,PAID,STUD29324,1,0,1,...,0,0,0,1,0,11,8,2017,14,4


In [118]:
df_merge_full['created_at'] = pd.to_datetime(df_merge_full['created_at'])

df_merge_full['day'] = df_merge_full['created_at'].dt.day
df_merge_full['month'] = df_merge_full['created_at'].dt.month
df_merge_full['year'] = df_merge_full['created_at'].dt.year
df_merge_full['hour'] = df_merge_full['created_at'].dt.hour
df_merge_full['weekday'] = df_merge_full['created_at'].dt.weekday


df_merge_full = df_merge_full.drop(columns=['created_at'])

/var/folders/_c/8b17td616wg63_3tyjv1dhb80000gn/T/ipykernel_49521/2671573870.py:3: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/_c/8b17td616wg63_3tyjv1dhb80000gn/T/ipykernel_49521/2671573870.py:4: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`

/var/folders/_c/8b17td616wg63_3tyjv1dhb80000gn/T/ipykernel_49521/2671573870.py:5: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [119]:
df_modeling = df_merge_full.drop(columns=['id', 'changed_at', 'user','batch','state','user_id'])
df_modeling['credits_category'] = pd.cut(df_modeling['credits'], bins=[500, 600, 1000, max(df_modeling['credits'])], labels=['500-600', '601-1000', '1001+'])
df_modeling = df_modeling.drop(columns=['credits'])

fig = px.bar(pd.DataFrame(df_modeling['credits_category'].value_counts().reset_index()), x='credits_category', y='count', title='Credits Category Distribution')
fig.show()

/var/folders/_c/8b17td616wg63_3tyjv1dhb80000gn/T/ipykernel_49521/247376658.py:2: PerformanceWarning:

DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`



# Modeling

In [120]:
y = df_modeling['credits_category']
X = df_modeling.drop(columns=['credits_category'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)

In [121]:
model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.1,
    depth=6,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    verbose=50
)

model.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	learn: 0.6761961	test: 0.6912159	best: 0.6912159 (0)	total: 125ms	remaining: 1m 2s
50:	learn: 0.6756006	test: 0.6922084	best: 0.6922084 (1)	total: 2.71s	remaining: 23.9s
100:	learn: 0.6839389	test: 0.6920596	best: 0.6926551 (85)	total: 5.02s	remaining: 19.8s
150:	learn: 0.6974390	test: 0.6930025	best: 0.6930521 (143)	total: 7.32s	remaining: 16.9s
200:	learn: 0.7244392	test: 0.6930521	best: 0.6935484 (159)	total: 9.47s	remaining: 14.1s
250:	learn: 0.7512408	test: 0.6925062	best: 0.6935484 (159)	total: 11.7s	remaining: 11.6s
300:	learn: 0.7857852	test: 0.6924566	best: 0.6935484 (159)	total: 14s	remaining: 9.24s
350:	learn: 0.8137780	test: 0.6911663	best: 0.6935484 (159)	total: 16.1s	remaining: 6.85s
400:	learn: 0.8415724	test: 0.6910670	best: 0.6935484 (159)	total: 18.5s	remaining: 4.56s
450:	learn: 0.8691682	test: 0.6889330	best: 0.6935484 (159)	total: 20.7s	remaining: 2.25s
499:	learn: 0.8935875	test: 0.6879901	best: 0.6935484 (159)	total: 23s	remaining: 0us

bestTest = 0.6935483871

In [130]:
# Get importance values
importances = model.get_feature_importance()
feature_names = X_train.columns

# Combine into DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

fig = px.bar(feature_importance_df.head(10), x='Importance', y='Feature', title='Top 20 Feature Importances')
fig.show()

## PCA to reduce attributes - After using PCA it won't be interpretable as leaving as it is

In [122]:
# This doesn't work anymore - from the consultation with VS it is bad idea to do PCA as it lose the interpretability

"""from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

X = df_merge_full.drop(columns=['id', 'user', 'batch', 'state', 'user_id', 'changed_at', 'created_at'])

# PCA is sensitive to scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=0.95) # Retain 95% of variance
X_pca = pca.fit_transform(X_scaled)

print(f"Original shape: {X.shape}")
print(f"Reduced shape: {X_pca.shape}")"""

'from sklearn.decomposition import PCA\nfrom sklearn.preprocessing import StandardScaler\nimport pandas as pd\n\nX = df_merge_full.drop(columns=[\'id\', \'user\', \'batch\', \'state\', \'user_id\', \'changed_at\', \'created_at\'])\n\n# PCA is sensitive to scaling\nscaler = StandardScaler()\nX_scaled = scaler.fit_transform(X)\n\npca = PCA(n_components=0.95) # Retain 95% of variance\nX_pca = pca.fit_transform(X_scaled)\n\nprint(f"Original shape: {X.shape}")\nprint(f"Reduced shape: {X_pca.shape}")'

In [140]:
df_atlas

,user_id,all_intro-2022,atlas_vzorek-2023,atlas_vzorek-2024,all_intro-trendaro_panel_all,sex-woman,sex-man,age-15_17,age-18_24,age-25_34,...,check-tv,check-radio,check-household,check-overdraft,check-naramek,isic2_age-21_26,isic2_age-15_20,isic_age-15_17,isic_age-18_22,isic_age-23_26
0,STUD29866,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,1,1,0,0
1,STUD35256,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,STUD29684,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,STUD11967,0,0,0,1,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,1
4,STUD57644,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43528,STUD33520,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43529,STUD64071,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43530,STUD82916,1,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
43531,STUD63130,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
